In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [2]:
# Load Dataset
df = pd.read_csv(r'C:\Users\srfzx\Downloads\ECG5000\ECG5000_TRAIN.csv', nrows=50000)
df2 = pd.read_csv(r'C:\Users\srfzx\Downloads\ECG5000\ECG5000_TEST.csv', nrows=50000)
df.shape


(500, 141)

In [3]:
y_train = df.target
X_train = df.drop(['target'],axis=1)
y_test = df2.target
X_test = df2.drop(['target'],axis=1)
X_test = df2.drop(['id'],axis=1)
X_test = X_test.drop(['target'],axis=1)

In [8]:
y_train.shape

(500,)

### Important

In all feature selection procedures, it is good practice to select the features by examining only the training set. And this is to avoid overfit.

In [7]:
df =df.drop(['id'], axis=1)

In [8]:
df.shape

(4217, 141)

In [9]:
# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(labels=['target'], axis=1),
    df['target'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((2951, 140), (1266, 140))

In [9]:
X_train.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att131,att132,att133,att134,att135,att136,att137,att138,att139,att140
0,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,-1.818286,-1.250522,-0.477492,...,0.160348,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137
1,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.560327,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250
2,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490659,-1.183580,-0.394229,...,1.284825,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797
3,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.491173,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564
4,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,0.966606,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456


In [ ]:
# step forward feature selection
# Select 10 features based on optimal ROC_AUC scoring criteria
#y_train = label_binarize(y_train, classes=[1, 2, 3, 4, 5])
sfs1 = SFS(RandomForestClassifier(n_jobs=4), 
           k_features=20, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=3)

sfs1 = sfs1.fit(np.array(X_train.fillna(0)), y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.4s remaining:    0.0s


In [15]:
selected_feat= X_train.columns[list(sfs1.k_feature_idx_)]
selected_feat

Index(['att3', 'att5', 'att51', 'att64', 'att68', 'att74', 'att90', 'att91',
       'att108', 'att113'],
      dtype='object')

In [25]:
selected_feat


Index(['att3', 'att5', 'att51', 'att64', 'att68', 'att74', 'att90', 'att91',
       'att108', 'att113'],
      dtype='object')

In [27]:
df2

,0
0,att3
1,att5
2,att51
3,att64
4,att68
5,att74
6,att90
7,att91
8,att108
9,att113


In [44]:
def run_randomForests(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train set')
    pred = rf.predict_proba(X_train)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

In [45]:
# evaluate performance of algorithm built
# using selected features

run_randomForests(X_train[selected_feat].fillna(0),
                  X_test[selected_feat].fillna(0),
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.9998569308255675
Test set
Random Forests roc-auc: 0.9999166953306389
